# Day 10

## GENERIC SETUP

In [19]:
# General imports
import pytest
import ipytest
import time
import functools

# Setup ipytest
ipytest.autoconfig()

# Setup nb_black
%load_ext nb_black

# Decorator to time solutions
def timer(func):
    """
    Wrapper function.
    Print the runtime of the decorated function.
    """

    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()  # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()  # 2
        run_time = end_time - start_time  # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value

    return wrapper_timer

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

## SOLUTION SETUP

In [37]:
# Solution-specific imports
from collections import Counter
from functools import reduce
import operator

<IPython.core.display.Javascript object>

#### I/O functions

In [38]:
# What day do we solve? Used to identify the input datafile, integer value
DAY = 10


def get_input():
    with open(f"../data/{DAY}.txt", "r") as f:
        return split_input(f.read())


def split_input(input_raw):
    """Strip trailing newline, then split on newline, cast to int"""
    return [int(line.strip()) for line in input_raw.rstrip().split("\n")]

<IPython.core.display.Javascript object>

#### Pytest input data

In [39]:
# Sample input
@pytest.fixture
def dummy_input_1():
    return """\
16
10
15
5
1
11
7
19
6
12
4
"""


@pytest.fixture
def dummy_input_2():
    return """\
28
33
18
42
31
14
46
20
48
47
24
23
49
45
19
38
39
11
1
32
25
35
8
17
7
9
4
2
34
10
3
"""

<IPython.core.display.Javascript object>

## Solution A

In [47]:
@timer
def solve_A(jolts):
    """
    1. Sort input
    2. Prepend 0 and append max+3
    3. Compute the differences from element i to element i+1
    4. Multiply how often each difference (1,2,3) occurs
    5. Return that product as result for A
    """
    # Sort input
    jolts.sort()

    # prepend 0 and append max(input)+3 to sorted input
    jolts_full = [0] + jolts + [max(jolts) + 3]

    # Calculate the differences between entries
    differences = [
        jolts_full[ix + 1] - jolts_full[ix] for ix, jolt in enumerate(jolts_full[:-1])
    ]

    # Count the occurrences of each offset 1, 2, or 3
    diff_count = Counter(differences)

    # Multiply the counts
    result = reduce(operator.mul, diff_count.values())

    # Return result
    return result

<IPython.core.display.Javascript object>

#### Tests

In [48]:
%%run_pytest[clean] -qq

def test_A_1(dummy_input_1):
    assert solve_A(split_input(dummy_input_1)) == 35
    
def test_A_2(dummy_input_2):
    assert solve_A(split_input(dummy_input_2)) == 220

<IPython.core.display.Javascript object>

..                                                                                                                                                                                                                                     [100%]


<IPython.core.display.Javascript object>

#### OUTPUT

In [49]:
solve_A(get_input())

Finished 'solve_A' in 0.0001 secs


1980

<IPython.core.display.Javascript object>

## Solution B

In [86]:
def calculate_options(sublist):
    """
    Count the valid configurations for the input list.
    
    Input: list of numbers.
    Output: Integer, number of valid configurations of list.
    """
    # If list is length <= 2, only one valid configuration possible
    # Because the last element is ALWAYS included and possible
    # (because it's necessary for the 3-offset jump in jolts)
    if len(sublist) <= 2:
        return 1

    # If length of list >= 3, for the next *3* numbers (and only if they're valid),
    # count the number of valid configurations, and sum the results.
    # Return that sum as the number of valid configurations for the sublist
    return sum(
        [
            calculate_options(sublist[ix + 1 :])
            for ix, val in enumerate(sublist[1:4])
            if (val - sublist[0]) <= 3
        ]
    )

@timer
def solve_B(jolts):
    """
    1. Sort input
    2. Prepend 0 and append max+3
    3. Compute the differences from element i to element i+1
    4. Generate sublists at each difference == 3 index
    5. Count the number of valid configurations for each sublist
    6. Multiply these possible configurations for each sublist with eachother
       to get the possible configurations for the sublists concatenated (= full list)
    7. Return the output of that product as result for B
    
    Input: List of numbers.
    Output: Integer.
    """
    # Sort input
    jolts.sort()

    # prepend 0 and append max(input)+3 to sorted input
    jolts_full = [0] + jolts + [max(jolts) + 3]

    # Calculate the differences between entries
    differences = [
        jolts_full[ix + 1] - jolts_full[ix] for ix, jolt in enumerate(jolts_full[:-1])
    ]
    
    # Initialisation of result calculation
    ix_low = 0
    result = 1
    
    # Create sublists (= subproblems) based on offset==3 splits
    for ix, val in enumerate(differences):
        if val == 3:
            result = result * calculate_options(jolts_full[ix_low: ix+1])
            ix_low = ix+1
            
            
    return result


<IPython.core.display.Javascript object>

#### Tests

In [87]:
%%run_pytest[clean] -qq

def test_B_1(dummy_input_1):
    assert solve_B(split_input(dummy_input_1)) == 8
    
def test_B_2(dummy_input_2):
    assert solve_B(split_input(dummy_input_2)) == 19208

<IPython.core.display.Javascript object>

..                                                                                                                                                                                                                                     [100%]


<IPython.core.display.Javascript object>

#### OUTPUT

In [78]:
solve_B(get_input())

Finished 'solve_B' in 0.0002 secs


4628074479616

<IPython.core.display.Javascript object>